# Top Docs for Wikipedia 50-topics models

In [1]:
import sys
import os
import pathlib
import numpy as np
import json
import pandas as pd
from scipy import sparse

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

if 'src.topics_docs_selection.doc_selector' in sys.modules:
    print("doc_selector already in modules")
    del sys.modules['src.topics_docs_selection.doc_selector']

from src.topics_docs_selection.doc_selector import DocSelector

In [2]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

In [3]:
def load_model_info(model_path):
    if model_path.stem == "mallet":
        thetas = np.load(model_path / "doctopics.npz.npy")
        betas = np.load(model_path / "beta.npy")
    elif model_path.stem == "ctm":
        thetas = np.load(model_path / "train.theta.npy")
        betas = np.load(model_path / "beta.npy")
    else:
        thetas = sparse.load_npz(pathlib.Path(model_path) / "thetas.npz").toarray()
        betas = np.load(model_path / "betas.npy")
    
    try:
        with open(model_path / "vocab.json") as infile:
            vocab_w2id = json.load(infile)
    except:
        vocab_w2id = {}
        with (pathlib.Path(model_path)/'vocab.txt').open('r', encoding='utf8') as fin:
            for i, line in enumerate(fin):
                wd = line.strip()
                vocab_w2id[wd] = i
    
    try:
        bow = np.load(model_path / "bow_mat.npy")
    except:
        bow = None
    
    vocab_id2w = dict(zip(vocab_w2id.values(), vocab_w2id.keys()))
    keys = [
        [vocab_id2w[idx]
            for idx in row.argsort()[::-1][:15]]
        for row in betas
    ]
        
    return thetas, betas, vocab_w2id, bow, keys

def create_top_docs_dataframe(method, top_docs, df, keys, thetas):
    return pd.DataFrame({
        "id_tpc": range(len(thetas.T)),
        "keys": keys,
        **{f"top doc {i+1}": [df.text.iloc[top_docs[method][k][i]][0:1000] for k in range(len(thetas.T))] for i in range(5)}
    })

def create_top_docs_thetas_dataframe(method, top_docs, df, keys, thetas):
    return pd.DataFrame({
        "id_tpc": range(len(thetas.T)),
        "keys": keys,
        **{f"top doc {i+1}": [thetas.T[k][top_docs[method][k][i]] for k in range(len(thetas.T))] for i in range(5)}
    })

def reduce_threshold(matrix, initial_threshold, decrement=0.01):
    threshold = initial_threshold
    while True:
        modified_matrix = matrix.copy()
        satisfied = True

        for col in range(matrix.shape[1]):
            if np.all(matrix[:, col] <= threshold):
                satisfied = False
                break
        
        if satisfied:
            break
        
        threshold -= decrement

    return threshold

In [8]:
mallet_path = pathlib.Path("../data/models/mallet")
ctm_path = pathlib.Path("../data/models/ctm")

path_save_tops = pathlib.Path("../data/top_docs")
corpus_path = pathlib.Path("../data/document-data/wikitext/processed/labeled/vocab_15k/train.metadata.jsonl")
text_column = "tokenized_text"

In [9]:
# Read documents
df = pd.read_json(corpus_path, lines=True)

df["id_top"] = range(len(df))
df["len"] = df[text_column].apply(lambda x: len(x.split()))
df["text_split"] = df[text_column].apply(lambda x: x.split())
corpus = df["text_split"].values.tolist()
#df.head()

In [10]:
methods = ['thetas', 'thetas_sample', 'thetas_thr']#'elbow', 'sall', 'spart', 's3'
n_top_words_s3 = 100
n_top_docs = 5

doc_selector = DocSelector()

## Mallet

In [11]:
thetas, betas, vocab_w2id, bow, keys = load_model_info(mallet_path)

In [12]:
max_thetas = np.max(thetas)
min_thetas = np.min(thetas)
print(f"Max/min thetas: {max_thetas} / {min_thetas}")
thr = (0.5,0.9)

Max/min thetas: 0.999596118927002 / 9.460147794015938e-07


In [13]:
top_docs = {
    method: doc_selector.get_top_docs(method=method, thetas=thetas, bow=bow, betas=betas, corpus=corpus, vocab_w2id=vocab_w2id,thr=thr, top_words = None, ntop=n_top_docs, model_path=mallet_path)[0]
    for method in methods
}

In [35]:
for method in top_docs.keys():
    print("##" * 50)
    print(f"-- -- METHOD: {method}")
    df_method = create_top_docs_thetas_dataframe(method, top_docs, df, keys, thetas)
    display(df_method.iloc[0:9])
    (path_save_tops /mallet_path.name).mkdir(exist_ok=True)
    this_path_save = path_save_tops / mallet_path.name / f"{method}_top_docs.xlsx"
    df_method.to_excel(this_path_save, index=False)
    print("\n")

####################################################################################################
-- -- METHOD: thetas


,id_tpc,keys,top doc 1,top doc 2,top doc 3,top doc 4,top doc 5
0,0,"[ship, ships, war, german, fleet, guns, tons, british, torpedo, long, class, cruiser, service, italian, mph]",0.998030,0.997481,0.997357,0.997273,0.997177
1,1,"[british, ships, french, ship, fleet, spanish, island, captain, admiral, naval, port, coast, sea, battle, crew]",0.989845,0.986242,0.975379,0.974118,0.972367
2,2,"[horse, horses, white, breed, dog, black, breeds, dogs, breeding, known, animals, century, riding, type, bred]",0.994985,0.982282,0.976747,0.929790,0.926588
3,3,"[match, australia, runs, test, innings, england, cricket, team, wickets, scored, second, matches, took, australian, played]",0.996480,0.995988,0.986190,0.984264,0.982386
4,4,"[station, line, railway, bridge, trains, new, service, construction, built, train, services, stations, opened, tunnel, london]",0.995329,0.992506,0.986704,0.985714,0.983889
5,5,"[species, shark, long, sharks, females, fish, found, birds, small, large, males, range, bird, brown, white]",0.998083,0.997636,0.997168,0.996253,0.995912
6,6,"[police, said, death, prison, murder, case, found, trial, people, killed, later, evidence, arrested, reported, court]",0.824423,0.762204,0.756807,0.754640,0.744599
7,7,"[world, won, games, olympics, time, olympic, event, round, team, final, gold, medal, summer, events, record]",0.995232,0.994828,0.994677,0.994189,0.994031
8,8,"[aircraft, air, flight, wing, squadron, flying, fighter, pilot, mission, engine, ground, production, crew, pilots, flew]",0.968833,0.963443,0.958630,0.952006,0.951871




####################################################################################################
-- -- METHOD: thetas_thr


,id_tpc,keys,top doc 1,top doc 2,top doc 3,top doc 4,top doc 5
0,0,"[ship, ships, war, german, fleet, guns, tons, british, torpedo, long, class, cruiser, service, italian, mph]",0.899761,0.896811,0.894165,0.893444,0.892741
1,1,"[british, ships, french, ship, fleet, spanish, island, captain, admiral, naval, port, coast, sea, battle, crew]",0.893820,0.888176,0.879852,0.877769,0.875201
2,2,"[horse, horses, white, breed, dog, black, breeds, dogs, breeding, known, animals, century, riding, type, bred]",0.897272,0.897051,0.891696,0.891580,0.891322
3,3,"[match, australia, runs, test, innings, england, cricket, team, wickets, scored, second, matches, took, australian, played]",0.899576,0.893354,0.892173,0.891980,0.889254
4,4,"[station, line, railway, bridge, trains, new, service, construction, built, train, services, stations, opened, tunnel, london]",0.897884,0.897677,0.896002,0.894659,0.894405
5,5,"[species, shark, long, sharks, females, fish, found, birds, small, large, males, range, bird, brown, white]",0.899338,0.898731,0.896419,0.896383,0.889896
6,6,"[police, said, death, prison, murder, case, found, trial, people, killed, later, evidence, arrested, reported, court]",0.824423,0.762204,0.756807,0.754640,0.744599
7,7,"[world, won, games, olympics, time, olympic, event, round, team, final, gold, medal, summer, events, record]",0.898961,0.898423,0.898331,0.897278,0.895956
8,8,"[aircraft, air, flight, wing, squadron, flying, fighter, pilot, mission, engine, ground, production, crew, pilots, flew]",0.898050,0.893859,0.893809,0.890589,0.887420


In [36]:
for method in top_docs.keys():
    print("##" * 50)
    print(f"-- -- METHOD: {method}")
    df_method = create_top_docs_dataframe(method, top_docs, df, keys, thetas)
    display(df_method.iloc[0:9])
    (path_save_tops / mallet_path.name).mkdir(exist_ok=True)
    this_path_save = path_save_tops / mallet_path.name / f"{method}_top_docs.xlsx"
    df_method.to_excel(this_path_save, index=False)
    print("\n")

####################################################################################################
-- -- METHOD: thetas


,id_tpc,keys,top doc 1,top doc 2,top doc 3,top doc 4,top doc 5
0,0,"[ship, ships, war, german, fleet, guns, tons, british, torpedo, long, class, cruiser, service, italian, mph]","Bremen-class cruiser = The Bremen class was a group of seven light cruisers built for the Imperial German Navy in the early 1900s . The seven ships , Bremen , Hamburg , Berlin , Lübeck , München , Leipzig , and Danzig , were an improvement upon the previous Gazelle class . They were significantly larger than the earlier class , and were faster and better armored . Like the Gazelles , they were armed with a main battery of ten 10.5 cm SK L / 40 guns and a pair of torpedo tubes . The ships of the Bremen class served in a variety of roles , from overseas cruiser to fleet scout to training ship . Bremen and Leipzig were deployed to the American and Asian stations , respectively , while the other five ships remained in German waters with the High Seas Fleet . At the outbreak of World War I in August 1914 , Leipzig was in the Pacific Ocean in the East Asia Squadron ; she saw action at the Battle of Coronel in November and was sunk a month later at the Battle of the Falkland Islands . Bremen","Italian battleship Emanuele Filiberto = The Emanuele Filiberto was a pre-dreadnought battleship built for the Italian Navy ( Italian : Regia Marina ) during the 1890s . Her keel was laid down in October 1893 and she was launched in September 1897 ; work was completed in April 1902 . She had one sister ship , Ammiraglio di Saint Bon , the lead ship of the Ammiraglio di Saint Bon class . She was armed with a main battery of four 10-inch ( 254 mm ) guns and was capable of a speed in excess of 18 knots ( 33 km / h ; 21 mph ) . Emanuele Filiberto served in the active squadron of the Italian navy for the first several years of her career . She was assigned to the 3rd Division during the Italo-Turkish War of 1911 – 1912 . During the war , she was involved in the assaults on Tripoli in North Africa and on the island of Rhodes in the eastern Mediterranean Sea . She was obsolescent by World War I and was slated to be broken up in 1914 – 15 , but the need for warships granted Emanuele Filiberto a","SMS Graudenz = SMS Graudenz was the lead ship of her class of light cruisers . She had one sister ship , SMS Regensburg . The ship was built by the German Kaiserliche Marine ( Imperial Navy ) in the Kaiserliche Werft shipyard in Kiel , laid down in 1912 and commissioned into the High Seas Fleet in August 1914 , days after the outbreak of World War I. She was named for the then-German town of Graudenz ( now Grudziądz , Poland ) . The ship was armed with a main battery of twelve 10.5 cm SK L / 45 guns and had a top speed of 27.5 knots ( 50.9 km / h ; 31.6 mph ) . Graudenz saw extensive service during World War I , including serving as part of the reconnaissance screen for the battlecruisers of the I Scouting Group during the raid on Scarborough , Hartlepool and Whitby in December 1914 . The ship also took part in the Battle of Dogger Bank in January 1915 , and the Battle of the Gulf of Riga in August 1915 . She had been damaged by a mine and was unable to participate in the Battle of Jut","Graudenz-class cruiser = The Graudenz class of light cruisers was a class of two ships built for the German Imperial Navy . The class comprised SMS Graudenz and SMS Regensburg . The ships both were laid down in 1912 , launched in October 1913 and April 1914 and commissioned in August 1914 and January 1915 , respectively . They were armed with a main battery of twelve 10.5 cm guns , though over the course of their careers , they were rearmed with seven more powerful 15 cm guns . They displaced 6,382 t ( 6,281 long tons ) at full load and were rated at a top speed of 27.5 knots ( 50.9 km / h ; 31.6 mph ) Both ships saw extensive service during World War I , primarily in the reconnaissance forces of the High Seas Fleet . They participated in several raids on the British coast , screening for the battlecru



####################################################################################################
-- -- METHOD: thetas_thr


,id_tpc,keys,top doc 1,top doc 2,top doc 3,top doc 4,top doc 5
0,0,"[ship, ships, war, german, fleet, guns, tons, british, torpedo, long, class, cruiser, service, italian, mph]","Italian ironclad Sardegna = Sardegna was the third of three Re Umberto-class ironclad battleships built for the Italian Regia Marina ( Royal Navy ) . The ship , named for the island of Sardinia , was laid down in La Spezia in October 1885 , launched in September 1890 , and completed in February 1895 . She was armed with a main battery of four 13.5-inch ( 340 mm ) guns and had a top speed of 20.3 knots ( 37.6 km / h ; 23.4 mph ) — albeit at the cost of armor protection – and she was one of the first warships to be equipped with a wireless telegraph . Sardegna spent the first decade of her career in the Active Squadron of the Italian fleet . Thereafter , she was transferred to the Reserve Squadron , and by 1911 , she was part of the Training Division . She took part in the Italo-Turkish War of 1911 – 12 , where she escorted convoys to North Africa and supported Italian forces ashore by bombarding Ottoman troops . During World War I , Sardegna served as the flagship of the naval forces de","Italian battleship Napoli = Napoli was a Regina Elena-class pre-dreadnought battleship built for the Italian Regia Marina ( Royal Navy ) in 1903 – 08 . She was the last member of the four-ship class , which included the lead ship Regina Elena , Vittorio Emanuele , and Roma . Napoli was armed with a main battery of two 12-inch ( 305 mm ) and twelve 8 in ( 203 mm ) guns , and was capable of a top speed of 21 knots ( 39 km / h ; 24 mph ) . Napoli saw action in the Italo-Turkish War in 1911 and 1912 ; she took part in the attack on Derna , Libya , and the amphibious assaults on the islands of Rhodes and the Dodecanese in the Aegean Sea . Napoli remained in service during World War I in 1915 – 18 , but saw no action as a result of the cautious policies of both the Italian and Austro-Hungarian navies . She remained in the Italian inventory until she was stricken from the naval register in August 1926 and was subsequently broken up for scrap . = = Design = = Napoli was 144.6 meters ( 474 ft )","SM UB-3 = SM UB-3 was a German Type UB I submarine or U-boat in the German Imperial Navy ( German : Kaiserliche Marine ) during World War I. She disappeared on her first patrol in May 1915 , and was the first of her class to be lost . UB-3 was ordered in October 1914 and was laid down at the Germaniawerft shipyard in Kiel in November . UB-3 was a little more than 28 metres ( 92 ft ) in length and displaced between 127 and 142 metric tons ( 125 and 140 long tons ) , depending on whether surfaced or submerged . She carried two torpedoes for her two bow torpedo tubes and was also armed with a deck-mounted machine gun . She was launched and commissioned as SM UB-3 in March 1915 . UB-3 was broken into sections and shipped by rail to the Austro-Hungarian port of Pola in April for reassembly . She officially joined the Pola Flotilla on 1 May and departed on her first patrol for temporary duty in Turkey on 23 May , and was never seen again . A postwar German study concluded that UB-3 was likel","Italian cruiser Coatit = Coatit was a torpedo cruiser of the Italian Regia Marina built in the late 1890s . She was the second and final member of the Agordat class . The ship , which was armed with twelve 76 mm ( 3.0 in ) guns and two 450 mm ( 18 in ) torpedo tubes , was too slow and short-ranged to be able to scout effectively for the fleet , so her career was limited . She saw action during the Italo-Turkish War in 1911 – 12 , where she provided gunfire support to Italian troops in North Africa . She also caused a minor diplomatic incident from an attack on retreating Ottoman soldiers in Anatolia . Coatit was part of an international fleet sent to Constantinople when the city appeared to be at risk of falling to the Bulgarian Army during the First Balkan War . In 1919 , she was converted into a min

## CTM

In [37]:
thetas, betas, vocab_w2id, bow, keys = load_model_info(ctm_path)

In [38]:
max_thetas = np.max(thetas)
min_thetas = np.min(thetas)
print(f"Max/min thetas: {max_thetas} / {min_thetas}")
thr = (0.5,0.9)

thr_list = list(thr)

thr_list[1] = reduce_threshold(thetas, thr_list[1], 0.05)
print("Updated thr_list:", thr_list)

# Adjust thr[0] to be at least 0.2 less than thr[1]
while thr_list[0] >= thr_list[1] - 0.2:
    thr_list[0] -= 0.01  
    if thr_list[0] <= 0:
        thr_list[0] = 0
        break

thr = tuple(thr_list)
print("Final thr:", thr)
print("Final thr[0]:", thr[0])

Max/min thetas: 0.9954009354114532 / 1.8919544385198605e-05
Updated thr_list: [0.5, 0.29999999999999977]
Final thr: (0.09999999999999969, 0.29999999999999977)
Final thr[0]: 0.09999999999999969


In [39]:
top_docs = {
    method: doc_selector.get_top_docs(method=method, thetas=thetas, bow=bow, betas=betas, corpus=corpus, vocab_w2id=vocab_w2id,thr=thr, top_words = None, ntop=n_top_docs, model_path=ctm_path)[0]
    for method in methods
}

In [40]:
for method in top_docs.keys():
    print("##" * 50)
    print(f"-- -- METHOD: {method}")
    df_method = create_top_docs_thetas_dataframe(method, top_docs, df, keys, thetas)
    display(df_method.iloc[0:9])
    (path_save_tops / mallet_path.name).mkdir(exist_ok=True)
    this_path_save = path_save_tops / mallet_path.name / f"{method}_top_docs.xlsx"
    df_method.to_excel(this_path_save, index=False)
    print("\n")

####################################################################################################
-- -- METHOD: thetas


,id_tpc,keys,top doc 1,top doc 2,top doc 3,top doc 4,top doc 5
0,0,"[credits, composition, allmusic, listing, charts, piano, drums, beats, liner, label, recording, download, ballad, songwriter, vocals]",0.329688,0.326009,0.324521,0.320877,0.315103
1,1,"[film, release, released, million, story, scene, scenes, script, effects, production, opening, plot, filming, original, director]",0.501727,0.498330,0.490960,0.483859,0.473820
2,2,"[career, hit, games, season, league, baseball, major_league_baseball, signed, home, played, manager, professional, minor, earned, hits]",0.737660,0.723641,0.714612,0.708765,0.705220
3,3,"[work, life, wrote, writing, published, book, written, women, works, history, world, writer, woman, early, books]",0.402178,0.376136,0.367494,0.365637,0.362293
4,4,"[tons, beam, deck, consisted, guns, laid, horsepower, armor, hull, boilers, displaced, steam, mounted, armament, battery]",0.421570,0.419186,0.410072,0.399300,0.389479
5,5,"[different, called, given, form, example, point, elements, real, use, possible, formula, sequence, particular, result, simple]",0.885582,0.860844,0.831554,0.822141,0.818491
6,6,"[war, american, later, wrote, years, began, men, time, british, land, united_states, known, general, early, received]",0.448018,0.428517,0.388362,0.384286,0.383984
7,7,"[game, released, games, player, music, version, original, series, release, story, characters, players, main, character, japan]",0.657687,0.651847,0.594150,0.569258,0.553545
8,8,"[tropical, storm, cyclone, area, cyclones, near, level, wind, low, formed, depression, miles, winds, water, west]",0.421496,0.418954,0.406579,0.401785,0.392336




####################################################################################################
-- -- METHOD: thetas_thr


,id_tpc,keys,top doc 1,top doc 2,top doc 3,top doc 4,top doc 5
0,0,"[credits, composition, allmusic, listing, charts, piano, drums, beats, liner, label, recording, download, ballad, songwriter, vocals]",0.298118,0.298068,0.297612,0.296216,0.295444
1,1,"[film, release, released, million, story, scene, scenes, script, effects, production, opening, plot, filming, original, director]",0.299881,0.298767,0.297957,0.296576,0.296187
2,2,"[career, hit, games, season, league, baseball, major_league_baseball, signed, home, played, manager, professional, minor, earned, hits]",0.299455,0.296663,0.292269,0.291120,0.290753
3,3,"[work, life, wrote, writing, published, book, written, women, works, history, world, writer, woman, early, books]",0.297983,0.296374,0.295663,0.293647,0.292484
4,4,"[tons, beam, deck, consisted, guns, laid, horsepower, armor, hull, boilers, displaced, steam, mounted, armament, battery]",0.299092,0.298865,0.298346,0.297985,0.297860
5,5,"[different, called, given, form, example, point, elements, real, use, possible, formula, sequence, particular, result, simple]",0.299226,0.298007,0.293263,0.292882,0.289715
6,6,"[war, american, later, wrote, years, began, men, time, british, land, united_states, known, general, early, received]",0.299785,0.298510,0.298420,0.297476,0.295700
7,7,"[game, released, games, player, music, version, original, series, release, story, characters, players, main, character, japan]",0.299660,0.299135,0.297911,0.297547,0.297229
8,8,"[tropical, storm, cyclone, area, cyclones, near, level, wind, low, formed, depression, miles, winds, water, west]",0.299137,0.298311,0.290756,0.290666,0.289036


In [41]:
for method in top_docs.keys():
    print("##" * 50)
    print(f"-- -- METHOD: {method}")
    df_method = create_top_docs_dataframe(method, top_docs, df, keys, thetas)
    display(df_method.iloc[0:9])
    (path_save_tops / ctm_path.name).mkdir(exist_ok=True)
    this_path_save = path_save_tops / ctm_path.name / f"{method}_top_docs.xlsx"
    df_method.to_excel(this_path_save, index=False)
    print("\n")

####################################################################################################
-- -- METHOD: thetas


,id_tpc,keys,top doc 1,top doc 2,top doc 3,top doc 4,top doc 5
0,0,"[credits, composition, allmusic, listing, charts, piano, drums, beats, liner, label, recording, download, ballad, songwriter, vocals]","Oh Blue Christmas = Oh Blue Christmas is an EP by the American band A Fine Frenzy , released in November 2009 in the United States by Virgin Records . It was initially released exclusively through Target just two months following the group 's previous studio album Bomb in a Birdcage . The EP contains cover versions of three popular holiday songs ( "" Blue Christmas "" , "" Winter Wonderland "" and "" Christmas Time Is Here "" from A Charlie Brown Christmas ) as well as three original tracks . According to Alison Sudol , A Fine Frenzy 's leader , the collection of songs was recorded in five days at record producer David Bianco 's studio in Los Angeles . The EP , and "" Christmas Time Is Here "" in particular , received positive critical reception overall . In the United States , Oh Blue Christmas peaked at number 87 on the Billboard 200 , number three on Billboard 's Top Holiday Albums chart and number 28 on the Top Rock Albums chart . A Fine Frenzy was touring throughout the United States arou","The Stripped Mixes = The Stripped Mixes ( released on CD for a limited time as The Motown 50 Mixes ) , is a collection of American recording artist and former Jackson 5 member Michael Jackson 's classic songs . The songs featured on the album are mainly from Jackson 's career as a member of the Jackson 5 from the 1960s to the 1980s ; songs credited to Jackson as a solo artist are from his albums during his Motown-era . Other songs included on the album are "" stripped "" mixes of Jackson material , meaning the songs are of a quieter tone and most of the drums have been removed . Following a surge in Jackson 's popularity after his death in June 2009 , it was confirmed on July 7 , that The Stripped Mixes would be released . The album was made available as a digital download on July 7 , 2009 , and as a CD on July 28 . A remixed song , entitled "" I 'll Be There ( Minus Mix ) "" , was released on iTunes to promote the album on June 9 , 2009 , prior to Jackson 's death . The Stripped Mixes was","A Different Kind of Love Song = "" A Different Kind of Love Song "" is a song by American recording artist Cher , taken from her 24th studio album , Living Proof ( 2002 ) . The song was written and produced by Sigurd Rosnes and Johan Aberg , with additional writing done by Michelle Lewis , and was co-produced by Anders Hansson . The dance-pop song alludes to themes of tragedy , heroism and brotherhood , and was released as a double A-side single with "" The Music 's No Good Without You "" in July 2002 through Warner Bros. Records and WEA . "" A Different Kind of Love Song "" received mostly positive reviews from music critics , who deemed it as one of the album 's highlights , although noting Cher 's heavily processed vocals due to the use of auto-tune . The song charted on a few Billboard components , such as Dance / Club Play Songs , where it reached number one , Hot Dance Music / Maxi-Singles Sales , peaking at number two , and on the Adult Contemporary chart . No accompanying music video","Electric ( Robyn song ) = "" Electric "" is a song by Swedish recording artist Robyn from her second studio album My Truth ( 1999 ) . It was released as the album 's lead single on 29 April 1999 by BMG Sweden . Robyn wrote the track in collaboration with its producers Ulf Lindström and Johan Ekhé . The single artwork features the singer wearing a feather headpiece designed by Sebastian Wahl . Musically , "" Electric "" is an electronic funk song , and the lyrics address unexpected life events that make one feel alive . "" Electric "" was hailed by music critics as a highlight on My Truth . It achieved commercial success in Sweden , with a peak position of number six on the Sverigetopplistan chart and a gold certification by the Swedish Recording Industry Asso



####################################################################################################
-- -- METHOD: thetas_thr


,id_tpc,keys,top doc 1,top doc 2,top doc 3,top doc 4,top doc 5
0,0,"[credits, composition, allmusic, listing, charts, piano, drums, beats, liner, label, recording, download, ballad, songwriter, vocals]","I 'm That Chick = "" I 'm That Chick "" is a song by American singer and songwriter Mariah Carey from her eleventh studio album , E = MC ² ( 2008 ) . A disco track with influences of R & B , it was written by Carey , Johntá Austin and production duo Stargate ( Mikkel S. Eriksen and Tor E. Hermansen ) . It contains elements of a song called "" Off the Wall "" originally performed by Michael Jackson . Rod Temperton , the songs composer , received a songwriting for "" I 'm That Chick "" as result . The track was positively received amongst most music critics , many of whom called it the best song on the album and praised is disco composition . Carey performed "" I 'm That Chick "" on The Hills season premiere party , the 2008 Fashion Rocks , Canadian Idol , and included it on the set-list of her eighth concert tour , The Elusive Chanteuse Show ( 2014 ) . It peaked at number 82 on the U.S. Billboard Hot R & B / Hip-Hop Songs chart . = = Recording and composition = = "" I 'm That Chick "" was written","Sing Like Me = "" Sing Like Me "" is a song by American recording artist Chris Brown . It was released as a promotional single from his third studio album , Graffiti , on November 24 , 2009 in the United States . The number was written by Brown , Big Makk , Keith Thomas , Lorenza "" Big Lo "" Lennon and Atozzio Towns , and produced by Makk , Thomas and Lennon . A pop and R & B ballad , the song contains lyrics about Brown leaving a nightclub with several women . "" Sing Like Me "" received mixed response from critics , and charted for two weeks on the US Hot R & B / Hip-Hop Songs in 2010 , peaking at number eighty-four . = = Background and composition = = "" Sing Like Me "" was written by Chris Brown , Big Makk , Keith Thomas , andf Lorenza "" Big Lo "" Lennon , with Makk , Thomas and Lennon producing the track . It was recorded at The Compound — a recording studio in Orlando , Florida — and Tony Maserati mixed the tune at The Record Plant — a studio in Los Angeles , California . The cover art o","Hot Tottie = "" Hot Tottie "" is a song by recording artist Usher . It was written by Usher , Ester Dean , Jay-Z and Polow da Don , with the latter producing it . The song features guest vocals from rapper Jay-Z and background vocals by Ester Dean . It is the second single in the United States and Canada from his EP , Versus , which is an extension of his sixth studio album , Raymond v. Raymond . The song was sent to rhythmic and urban airplay on August 9 , 2010 . "" Hot Tottie "" samples Big Tymers 's "" Big Ballin ' "" off their 1998 album How You Luv That Vol . 2 . "" Hot Tottie "" is an R & B song with hip hop tones , accompanied by strobing , electronic beats . It received positive reviews , with many critics noting it as a standout from the EP . It peaked at number twenty-one on the US Billboard Hot 100 , and was a top ten hit on the US Hot R & B / Hip-Hop Songs chart . Usher performed the song on The Early Show and on his OMG Tour . = = Background and composition = = The song was leaked","I 'm Not Your Hero = "" I 'm Not Your Hero "" is a song written , recorded and performed by Canadian duo Tegan and Sara and produced by Greg Kurstin for the album Heartthrob ( 2013 ) . Recorded at Echo Studio and EastWest Studios in California in 2012 , it was released on October 21 of that year as a promotional single . The lyrics for the new wave synthpop ballad were written by Sara Quin , and discuss her teenage life in the 1990s when she was unable to identify with the lives of pop culture figures and had unforeseen thoughts about her later life . Guitar chords , drums , bass guitar , synths and harmonized vocals are also played in the track . "" I 'm Not Your Hero "" was mostly well-received by critics for its songwriting and composition . The 

## BERTopic

### Using default UMAP

In [12]:
bert_path=bert_path1
thetas, betas, vocab_w2id, bow, keys = load_model_info(bert_path)

In [13]:
max_thetas = np.max(thetas)
min_thetas = np.min(thetas)
print(f"Max/min thetas: {max_thetas} / {min_thetas}")
thr = (0.5,0.9)

Max/min thetas: 1.0 / 0.0


In [14]:
top_docs_per_topic = []

for doc_distr in thetas.T:
    sorted_docs_indices = np.argsort(doc_distr)[::-1]
    top = sorted_docs_indices[:5].tolist()
    top_docs_per_topic.append(top)
print(top_docs_per_topic[0])
print(thetas[top_docs_per_topic[0],0])
thetas_0 = thetas[:,0]
print(thetas_0[thetas_0 != 0])
print(len(thetas_0[thetas_0 != 0]), len(thetas_0))

[2048, 2057, 2159, 11874, 11837]
[1. 1. 1. 1. 1.]
[1.89075499e-01 1.30882240e-01 6.09278515e-02 6.67913372e-01
 3.38133812e-01 4.85862890e-01 3.30649412e-01 3.85559416e-02
 5.02076638e-01 6.12529423e-02 5.75834876e-02 2.17756821e-01
 1.30344060e-01 6.45948962e-02 2.25830817e-02 4.87116550e-01
 9.16618312e-02 2.49411818e-01 5.17975652e-02 5.60427221e-02
 2.31680329e-02 2.38416499e-01 1.00000000e+00 1.46257810e-01
 7.49744996e-02 1.86752557e-02 4.78921902e-02 2.65531704e-02
 1.12313736e-01 2.32354108e-02 3.26287881e-01 6.54536592e-01
 1.00000000e+00 6.92194924e-02 3.10522592e-02 3.67833967e-02
 1.07317387e-01 3.31774242e-01 2.26930923e-02 1.68831372e-02
 3.27395165e-01 1.98701724e-01 2.78372860e-02 3.41747059e-02
 3.81175311e-02 4.17466005e-01 1.73884639e-01 2.26930923e-02
 1.91190574e-01 5.20239332e-02 2.57535871e-01 4.12218490e-02
 4.78603608e-01 2.77853140e-01 1.00000000e+00 1.97979795e-01
 1.60302371e-01 1.00000000e+00 1.76875120e-01 5.26556459e-02
 2.18884116e-01 1.00000000e+00 1.89

In [15]:
max_thetas = np.max(thetas)
min_thetas = np.min(thetas)
print(f"Max/min thetas: {max_thetas} / {min_thetas}")
thr = (0.5,0.9)

top_docs_per_topic = []

for doc_distr in thetas.T:
    sorted_docs_indices = np.argsort(doc_distr)[::-1]
    top = sorted_docs_indices[:5].tolist()
    top_docs_per_topic.append(top)
print(top_docs_per_topic[0])
print(thetas[top_docs_per_topic[0],0])
thetas_0 = thetas[:,0]
print(thetas_0[thetas_0 != 0])

Max/min thetas: 1.0 / 0.0
[2048, 2057, 2159, 11874, 11837]
[1. 1. 1. 1. 1.]
[1.89075499e-01 1.30882240e-01 6.09278515e-02 6.67913372e-01
 3.38133812e-01 4.85862890e-01 3.30649412e-01 3.85559416e-02
 5.02076638e-01 6.12529423e-02 5.75834876e-02 2.17756821e-01
 1.30344060e-01 6.45948962e-02 2.25830817e-02 4.87116550e-01
 9.16618312e-02 2.49411818e-01 5.17975652e-02 5.60427221e-02
 2.31680329e-02 2.38416499e-01 1.00000000e+00 1.46257810e-01
 7.49744996e-02 1.86752557e-02 4.78921902e-02 2.65531704e-02
 1.12313736e-01 2.32354108e-02 3.26287881e-01 6.54536592e-01
 1.00000000e+00 6.92194924e-02 3.10522592e-02 3.67833967e-02
 1.07317387e-01 3.31774242e-01 2.26930923e-02 1.68831372e-02
 3.27395165e-01 1.98701724e-01 2.78372860e-02 3.41747059e-02
 3.81175311e-02 4.17466005e-01 1.73884639e-01 2.26930923e-02
 1.91190574e-01 5.20239332e-02 2.57535871e-01 4.12218490e-02
 4.78603608e-01 2.77853140e-01 1.00000000e+00 1.97979795e-01
 1.60302371e-01 1.00000000e+00 1.76875120e-01 5.26556459e-02
 2.188841

In [16]:
thetas_7 = thetas[:,7]
print(thetas_7[thetas_7 != 0])

[]


In [17]:
methods = ['thetas', 'thetas_thr']#'elbow' #'sall', 'spart', 's3'

top_docs = {
    method: doc_selector.get_top_docs(method=method, thetas=thetas, bow=bow, betas=betas, corpus=corpus, vocab_w2id=vocab_w2id,thr=thr, top_words = None, ntop=n_top_docs, model_path=bert_path)[0]
    for method in methods
}

In [18]:
for method in top_docs.keys():
    print("##" * 50)
    print(f"-- -- METHOD: {method}")
    df_method = create_top_docs_thetas_dataframe(method, top_docs, df, keys, thetas)
    display(df_method.iloc[0:9])
    (path_save_tops / bert_path.name).mkdir(exist_ok=True)
    this_path_save = path_save_tops / mallet_path.name / f"{method}_top_docs.xlsx"
    df_method.to_excel(this_path_save, index=False)
    print("\n")

####################################################################################################
-- -- METHOD: thetas


,id_tpc,keys,top doc 1,top doc 2,top doc 3,top doc 4,top doc 5
0,0,"[album, song, madonna, chart, video, music, band, songs, harrison, carey, track, pop, recording, vocals, lyrics]",1.00000,1.000000,1.000000,1.000000,1.000000
1,1,"[ship, guns, ships, tons, torpedo, knots, inch, cruiser, fleet, gun, deck, admiral, cruisers, turrets, german]",1.00000,1.000000,1.000000,1.000000,1.000000
2,2,"[episode, mulder, scully, doctor, episodes, trek, series, enterprise, character, viewers, files, amy, television, fringe, scene]",0.49536,0.492988,0.492387,0.487154,0.487154
3,3,"[game, player, gameplay, games, players, playstation, nintendo, released, graphics, characters, soundtrack, xbox, mode, version, manga]",1.00000,1.000000,0.504335,0.296402,0.233116
4,4,"[innings, team, runs, match, league, nba, baseball, batting, season, career, scored, wickets, championship, cricket, basketball]",1.00000,1.000000,0.886863,0.530042,0.519001
5,5,"[highway, route, road, freeway, interchange, intersection, terminus, north, east, continues, lane, state, avenue, passes, crosses]",1.00000,1.000000,1.000000,1.000000,1.000000
6,6,"[tropical, storm, hurricane, winds, depression, cyclone, mph, rainfall, damage, landfall, utc, wind, flooding, weakened, intensity]",1.00000,1.000000,1.000000,1.000000,1.000000
7,7,"[film, films, disney, movie, animated, role, comedy, simpsons, starred, cast, character, production, script, actor, director]",0.00000,0.000000,0.000000,0.000000,0.000000
8,8,"[species, shark, genus, sharks, cap, fruit, stem, fungus, spores, brown, nest, females, habitat, eggs, fin]",1.00000,1.000000,1.000000,1.000000,1.000000




####################################################################################################
-- -- METHOD: thetas_thr


,id_tpc,keys,top doc 1,top doc 2,top doc 3,top doc 4,top doc 5
0,0,"[album, song, madonna, chart, video, music, band, songs, harrison, carey, track, pop, recording, vocals, lyrics]",0.667913,0.666272,0.654537,0.596030,0.509474
1,1,"[ship, guns, ships, tons, torpedo, knots, inch, cruiser, fleet, gun, deck, admiral, cruisers, turrets, german]",0.899718,0.899627,0.899611,0.899185,0.899133
2,2,"[episode, mulder, scully, doctor, episodes, trek, series, enterprise, character, viewers, files, amy, television, fringe, scene]",0.000000,0.000000,0.000000,0.000000,0.000000
3,3,"[game, player, gameplay, games, players, playstation, nintendo, released, graphics, characters, soundtrack, xbox, mode, version, manga]",0.504335,0.000000,0.000000,0.000000,0.000000
4,4,"[innings, team, runs, match, league, nba, baseball, batting, season, career, scored, wickets, championship, cricket, basketball]",0.886863,0.530042,0.519001,0.515383,0.505194
5,5,"[highway, route, road, freeway, interchange, intersection, terminus, north, east, continues, lane, state, avenue, passes, crosses]",0.899802,0.896187,0.893901,0.892122,0.887142
6,6,"[tropical, storm, hurricane, winds, depression, cyclone, mph, rainfall, damage, landfall, utc, wind, flooding, weakened, intensity]",0.899205,0.898431,0.892474,0.889392,0.889129
7,7,"[film, films, disney, movie, animated, role, comedy, simpsons, starred, cast, character, production, script, actor, director]",0.000000,0.000000,0.000000,0.000000,0.000000
8,8,"[species, shark, genus, sharks, cap, fruit, stem, fungus, spores, brown, nest, females, habitat, eggs, fin]",0.719091,0.674786,0.672336,0.669706,0.664072


In [19]:
for method in top_docs.keys():
    print("##" * 50)
    print(f"-- -- METHOD: {method}")
    df_method = create_top_docs_dataframe(method, top_docs, df, keys, thetas)
    display(df_method.iloc[0:9])
    (path_save_tops / ctm_path.name).mkdir(exist_ok=True)
    this_path_save = path_save_tops / ctm_path.name / f"{method}_top_docs.xlsx"
    df_method.to_excel(this_path_save, index=False)
    print("\n")

####################################################################################################
-- -- METHOD: thetas


,id_tpc,keys,top doc 1,top doc 2,top doc 3,top doc 4,top doc 5
0,0,"[album, song, madonna, chart, video, music, band, songs, harrison, carey, track, pop, recording, vocals, lyrics]","I Love You ( Miss Robot ) = "" I Love You ( Miss Robot ) "" is a song written , performed and produced by the Buggles , a duo of Trevor Horn and Geoff Downes , for their 1980 debut studio album The Age of Plastic . It was not released as a single . The song is , according to Downes , about "" being on the road and making love to someone you don 't really like "" , although music critics consider the song 's subject having to do with a robot . The song was performed live in 2010 , as part of the first performance of all the tracks from The Age of Plastic . = = Production = = "" I Love You "" is the fourth track of The Buggles ' debut studio album The Age of Plastic , although it was not released as a single . Plastic was recorded in 1979 , and was made on a budget of £ 60,000 . The backing track of "" I Love You "" was recorded at Virgin 's Town House in West London , with mixing and recording of vocals held at Sarm East Studios . Gary Langan mixed the song on a Sunday in 1979 , between 11 : 00","Causing a Commotion = "" Causing a Commotion "" is a song by American singer Madonna from the soundtrack album to the 1987 film Who 's That Girl . It was released as the album 's second single on August 25 , 1987 by Sire Records . Its Silver Screen Single Mix later appeared on the 1991 UK compilation EP The Holiday Collection . Written and produced by Madonna and Stephen Bray , the song was inspired by Madonna 's relationship with then husband Sean Penn , and his abusive and violent nature . Containing a dance-oriented , up-tempo groove , the song begins with the chorus and is accompanied by a four-note descending bassline and staccato chords in the verse . Since its release , the song has received mixed reviews by critics . It became a top-ten hit in the United States , Australia , Canada , Ireland , Italy , New Zealand , Sweden and the United Kingdom , and topped the Billboard U.S. dance chart . Madonna performed the song on the Who 's That Girl World Tour – which was transmitted via s","Move Like This = Move Like This is the seventh , and , to date , final studio album by American rock band The Cars , released on May 10 , 2011 . The album is their first since 1987 's Door to Door and features all of the original band members except for bassist and vocalist Benjamin Orr , who died of pancreatic cancer in 2000 . The album reached the top ten of the Billboard 200 and peaked at number 2 on the Billboard Top Rock Albums chart ; a single from the album , "" Sad Song "" , reached number 33 on the Billboard Rock Songs chart . Following the release of the album , the band launched an eleven-city tour of North America . = = Background = = Move Like This is the first reunion of The Cars to feature original lead singer , songwriter , rhythm guitarist , and co-producer Ric Ocasek since their 1988 split . In 1997 , Ocasek told a journalist that the band would never reunite : "" I 'm saying never and you can count on that . "" A partial reunion of the band occurred in 2005 when keyboard","King Creole = King Creole is a 1958 American musical drama film directed by Michael Curtiz and starring Elvis Presley , Carolyn Jones , and Walter Matthau . Produced by Hal B. Wallis and based on the 1952 novel A Stone for Danny Fisher by Harold Robbins , the film is about a nineteen-year-old who gets mixed up with crooks and involved with two women . Presley later indicated that of all the characters he portrayed throughout his acting career , the role of Danny Fisher in King Creole was his favorite . To make the film , Presley was granted a 60-day deferment from January to March 1958 for beginning his military service . Location shooting in New Orleans was delayed several times by crowds of fans attracted by the stars , particularly Presley . The film was released by Paramount Pict



####################################################################################################
-- -- METHOD: thetas_thr


,id_tpc,keys,top doc 1,top doc 2,top doc 3,top doc 4,top doc 5
0,0,"[album, song, madonna, chart, video, music, band, songs, harrison, carey, track, pop, recording, vocals, lyrics]","Mulder and Scully ( song ) = "" Mulder and Scully "" is a song by Catatonia , released as a single from their 1998 album , International Velvet . The song makes direct reference to fictional FBI special agents Fox Mulder ( David Duchovny ) and Dana Scully ( Gillian Anderson ) , the two main characters of the popular sci-fi TV series The X-Files who work on cases linked to the paranormal , called X-Files . In an interview Cerys Matthews , co-writer of the song , explained that while she was not a serious fan of the show , the basic premise of the series matched the conceit of what she was trying to express . "" Mulder and Scully "" was released as the second single from the band 's album International Velvet . Originally , it was supposed to be the first single , but was delayed due to circumstances beyond the band 's control . The song was Catatonia 's first single in the United States . "" Mulder and Scully "" became the group 's break-out hit and received a mixed to positive from the music","Let Yourself Go ( Green Day song ) = "" Let Yourself Go "" is a song by American punk rock band Green Day . Written by Billie Joe Armstrong and produced by Rob Cavallo and the band for their ninth studio album , ¡ Uno ! , the song was released as the third single from the album on September 5 , 2012 . However , the "" official live video "" of the song was revealed on August 1 , 2012 . The song was described to be "" more punk "" than that of the band 's previous two singles "" Oh Love "" and "" Kill the DJ "" , and was compared to their 1990 's material , such as Dookie and Insomniac . Its theme was described as "" the kind of pop-smart here 's-my-fucking-problem miniatures "" . It received positive critical reception and charted at # 2 at UK Rock Chart and # 19 at US Alternative Rock . "" Let Yourself Go "" was played at secret shows of the band and at the 2012 MTV Video Music Awards . = = Recording and release = = After "" Let Yourself Go "" was played at a ' secret New York Halloween show ' of the","Human Nature ( Madonna song ) = "" Human Nature "" is a song recorded by American singer Madonna for her sixth studio album Bedtime Stories ( 1994 ) . It was written as an answer song to her critics , who had panned her provocative image of the previous two years and Madonna 's release of sexually explicit works . Written and produced by Madonna and Dave Hall , with additional writing from Shawn McKenzie , Kevin McKenzie and Milo Deering , "" Human Nature "" includes a looping sample from Main Source 's 1994 track "" What You Need "" . The track was released on June 6 , 1995 , by Maverick Records as the fourth and final single from Bedtime Stories . Musically it is an R & B track where the sound of drums and the sample is heard looping throughout , with Madonna sarcastically asking rhetorical questions based on her real-life actions over the last two years . The song received mostly positive reviews from music critics who have later noted its anthemic and empowering nature . "" Human Nature ""","Impressive Instant = "" Impressive Instant "" is a song by American singer-songwriter Madonna from her 2000 studio album Music . Originally intended to be the fourth single of the album , the release was cancelled due to a disagreement between Madonna and her recording company . Finally Warner Bros. released it in the United States as a promotional single on September 18 , 2001 . Written and produced by Madonna and Mirwais Ahmadzaï , the track is bright and uplifting in its content and composition . It was the first song that Madonna and Ahmadzaï worked on and recorded . The producer had to work separately on his laptop to generate the sound elements which Madonna wanted in the song , since it was difficult to generate the music in the recording studio . "" Impressive

### Using CUML UMAP

In [20]:
bert_path=bert_path2
thetas, betas, vocab_w2id, bow, keys = load_model_info(bert_path)

In [21]:
max_thetas = np.max(thetas)
min_thetas = np.min(thetas)
print(f"Max/min thetas: {max_thetas} / {min_thetas}")
thr = (0.5,0.9)

Max/min thetas: 0.04361054696265283 / 0.007184803239867564


In [22]:
top_docs_per_topic = []

for doc_distr in thetas.T:
    sorted_docs_indices = np.argsort(doc_distr)[::-1]
    top = sorted_docs_indices[:5].tolist()
    top_docs_per_topic.append(top)
print(top_docs_per_topic[0])
print(thetas[top_docs_per_topic[0],0])
thetas_0 = thetas[:,0]
print(thetas_0[thetas_0 != 0])
print(len(thetas_0[thetas_0 != 0]), len(thetas_0))

[540, 495, 8243, 2084, 13476]
[0.04041473 0.04000435 0.03999578 0.0399508  0.0398978 ]
[0.02474606 0.02339766 0.02269029 ... 0.02721015 0.01974477 0.02209826]
14290 14290


In [23]:
max_thetas = np.max(thetas)
min_thetas = np.min(thetas)
print(f"Max/min thetas: {max_thetas} / {min_thetas}")
thr = (0.5,0.9)

top_docs_per_topic = []

for doc_distr in thetas.T:
    sorted_docs_indices = np.argsort(doc_distr)[::-1]
    top = sorted_docs_indices[:5].tolist()
    top_docs_per_topic.append(top)
print(top_docs_per_topic[0])
print(thetas[top_docs_per_topic[0],0])
thetas_0 = thetas[:,0]
print(thetas_0[thetas_0 != 0])

Max/min thetas: 0.04361054696265283 / 0.007184803239867564
[540, 495, 8243, 2084, 13476]
[0.04041473 0.04000435 0.03999578 0.0399508  0.0398978 ]
[0.02474606 0.02339766 0.02269029 ... 0.02721015 0.01974477 0.02209826]


In [24]:
thetas_7 = thetas[:,7]
print(thetas_7[thetas_7 != 0])

[0.02618923 0.02658657 0.02549466 ... 0.02865615 0.02297869 0.02206499]


In [25]:
methods = ['thetas', 'thetas_thr']#'elbow' #'sall', 'spart', 's3'

top_docs = {
    method: doc_selector.get_top_docs(method=method, thetas=thetas, bow=bow, betas=betas, corpus=corpus, vocab_w2id=vocab_w2id,thr=thr, top_words = None, ntop=n_top_docs, model_path=bert_path)[0]
    for method in methods
}

In [26]:
for method in top_docs.keys():
    print("##" * 50)
    print(f"-- -- METHOD: {method}")
    df_method = create_top_docs_thetas_dataframe(method, top_docs, df, keys, thetas)
    display(df_method.iloc[0:9])
    (path_save_tops / bert_path.name).mkdir(exist_ok=True)
    this_path_save = path_save_tops / mallet_path.name / f"{method}_top_docs.xlsx"
    df_method.to_excel(this_path_save, index=False)
    print("\n")

####################################################################################################
-- -- METHOD: thetas


,id_tpc,keys,top doc 1,top doc 2,top doc 3,top doc 4,top doc 5
0,0,"[album, song, chart, music, video, band, songs, madonna, track, pop, single, recording, lyrics, vocals, billboard]",0.040415,0.040004,0.039996,0.039951,0.039898
1,1,"[tropical, storm, hurricane, winds, depression, cyclone, mph, damage, rainfall, landfall, utc, wind, weakened, flooding, intensity]",0.042614,0.041731,0.040794,0.040680,0.040205
2,2,"[episode, mulder, scully, doctor, episodes, trek, files, series, character, enterprise, viewers, television, season, broadcast, star]",0.033775,0.033074,0.032900,0.032791,0.032776
3,3,"[ship, guns, ships, tons, torpedo, knots, cruiser, inch, fleet, cruisers, deck, gun, turrets, admiral, steam]",0.036563,0.036460,0.036237,0.036231,0.035917
4,4,"[species, genus, shark, females, prey, eggs, males, cap, sharks, fruit, stem, brown, habitat, birds, specimens]",0.039610,0.038553,0.038279,0.037508,0.037439
5,5,"[highway, route, road, interchange, freeway, intersection, east, north, terminus, lane, continues, passes, avenue, state, highways]",0.041630,0.041283,0.041202,0.040894,0.040872
6,6,"[game, player, gameplay, games, players, playstation, nintendo, released, xbox, characters, graphics, soundtrack, mode, version, mario]",0.034853,0.034395,0.034217,0.034195,0.034103
7,7,"[film, films, bond, disney, movie, role, actor, director, cast, filming, script, production, starred, grossing, batman]",0.036714,0.036224,0.035789,0.035715,0.035667
8,8,"[army, persian, byzantine, arab, syria, emperor, polish, muslim, byzantines, constantine, battle, hungary, ottoman, king, forces]",0.036533,0.035892,0.035526,0.035292,0.035254




####################################################################################################
-- -- METHOD: thetas_thr


,id_tpc,keys,top doc 1,top doc 2,top doc 3,top doc 4,top doc 5
0,0,"[album, song, chart, music, video, band, songs, madonna, track, pop, single, recording, lyrics, vocals, billboard]",0.024746,0.022098,0.019745,0.027210,0.026884
1,1,"[tropical, storm, hurricane, winds, depression, cyclone, mph, damage, rainfall, landfall, utc, wind, weakened, flooding, intensity]",0.014964,0.014953,0.017573,0.014574,0.012290
2,2,"[episode, mulder, scully, doctor, episodes, trek, files, series, character, enterprise, viewers, television, season, broadcast, star]",0.022008,0.020833,0.019656,0.028439,0.027565
3,3,"[ship, guns, ships, tons, torpedo, knots, cruiser, inch, fleet, cruisers, deck, gun, turrets, admiral, steam]",0.018416,0.020364,0.024527,0.014942,0.014068
4,4,"[species, genus, shark, females, prey, eggs, males, cap, sharks, fruit, stem, brown, habitat, birds, specimens]",0.021337,0.020798,0.017622,0.019489,0.018522
5,5,"[highway, route, road, interchange, freeway, intersection, east, north, terminus, lane, continues, passes, avenue, state, highways]",0.017011,0.019943,0.024169,0.017422,0.016142
6,6,"[game, player, gameplay, games, players, playstation, nintendo, released, xbox, characters, graphics, soundtrack, mode, version, mario]",0.023334,0.023849,0.022222,0.023143,0.023165
7,7,"[film, films, bond, disney, movie, role, actor, director, cast, filming, script, production, starred, grossing, batman]",0.026189,0.022065,0.022979,0.028656,0.029000
8,8,"[army, persian, byzantine, arab, syria, emperor, polish, muslim, byzantines, constantine, battle, hungary, ottoman, king, forces]",0.021394,0.022333,0.023381,0.021193,0.022308


In [27]:
for method in top_docs.keys():
    print("##" * 50)
    print(f"-- -- METHOD: {method}")
    df_method = create_top_docs_dataframe(method, top_docs, df, keys, thetas)
    display(df_method.iloc[0:9])
    (path_save_tops / ctm_path.name).mkdir(exist_ok=True)
    this_path_save = path_save_tops / ctm_path.name / f"{method}_top_docs.xlsx"
    df_method.to_excel(this_path_save, index=False)
    print("\n")

####################################################################################################
-- -- METHOD: thetas


,id_tpc,keys,top doc 1,top doc 2,top doc 3,top doc 4,top doc 5
0,0,"[album, song, chart, music, video, band, songs, madonna, track, pop, single, recording, lyrics, vocals, billboard]","Lo Mejor de Mí ( song ) = "" Lo Mejor de Mí "" ( "" All My Best "" ) is a song written and produced by Rudy Pérez and first recorded by Spanish singer Juan Ramon for his second studio album Por Haberte Amado Tanto ( 1990 ) . In the song , the protagonist tells his lover how he gave his best despite not meeting his lover 's expectation . In 1997 , Mexican recording artist Cristian Castro covered the song for his fifth studio album Lo Mejor de Mí which Pérez also produced and arranged . Castro 's version peaked at number-one on the Billboard Hot Latin Songs and the Billboard Latin Pop Songs charts in the United States . The song received a Billboard Latin Music Awards and a Lo Nuestro nomination for Pop Song of the Year . Pérez earned the American Society of Composers , Authors and Publishers award in the Pop / Ballad field . = = Background = = In 1990 , Spanish recording artist Juan Ramon released his second studio album Por Haberte Amado Tanto which was arranged and produced by Cuban-Ameri","Enamorada de Ti ( song ) = "" Enamorada de Ti "" ( English : "" In Love With You "" ) is a song recorded by American Tejano recording artist Selena and originally written for her second studio album , Ven Conmigo ( 1990 ) . The song was written by Selena y Los Dinos band member Pete Astudillo and her brother , A.B. Quintanilla III , and produced by Quintanilla . "" Enamorada de Ti "" is a freestyle song , a musical genre popular during the late 1980s . The recording was remixed by Juan Magan for the eponymous album in 2012 , a project headed by Humberto Gatica . Although the song was moderately successful during the early 1990s , it was exposed to a wider audience in the remix album Enamorada de Ti . Because of this , the song peaked at number 17 on the US Billboard Regional Mexican Digital Songs chart that year . In 1997 , the posthumously-released single "" Is It the Beat ? "" adopted the rap verse found in "" Enamorada de Ti "" . In the lyrics the protagonist sings to her love interest about","Hot Tottie = "" Hot Tottie "" is a song by recording artist Usher . It was written by Usher , Ester Dean , Jay-Z and Polow da Don , with the latter producing it . The song features guest vocals from rapper Jay-Z and background vocals by Ester Dean . It is the second single in the United States and Canada from his EP , Versus , which is an extension of his sixth studio album , Raymond v. Raymond . The song was sent to rhythmic and urban airplay on August 9 , 2010 . "" Hot Tottie "" samples Big Tymers 's "" Big Ballin ' "" off their 1998 album How You Luv That Vol . 2 . "" Hot Tottie "" is an R & B song with hip hop tones , accompanied by strobing , electronic beats . It received positive reviews , with many critics noting it as a standout from the EP . It peaked at number twenty-one on the US Billboard Hot 100 , and was a top ten hit on the US Hot R & B / Hip-Hop Songs chart . Usher performed the song on The Early Show and on his OMG Tour . = = Background and composition = = The song was leaked","Joyas Prestadas = Joyas Prestadas : Pop and Joyas Prestadas : Banda ( English : Borrowed Jewels ) are the twelfth and final studio albums released by American recording artist Jenni Rivera on November 21 , 2011 by Fonovisa Records . Joyas Prestadas consists of eleven cover versions , with the first album being recorded in Latin pop , while the second was recorded in banda . Both albums were produced by Enrique Martinez . According to Rivera , the songs she chose to cover were those she was enamored with while working as a cashier in a record store . It was her first production to include ballad recordings . Joyas Prestadas : Pop reached number one on the Mexican Albums Chart and number one on the Billboard Top Latin Albums chart in the United States while Joyas Prestadas : B



####################################################################################################
-- -- METHOD: thetas_thr


,id_tpc,keys,top doc 1,top doc 2,top doc 3,top doc 4,top doc 5
0,0,"[album, song, chart, music, video, band, songs, madonna, track, pop, single, recording, lyrics, vocals, billboard]","Cicely Mary Barker = Cicely Mary Barker ( 28 June 1895 – 16 February 1973 ) was an English illustrator best known for a series of fantasy illustrations depicting fairies and flowers . Barker 's art education began in girlhood with correspondence courses and instruction at the Croydon School of Art . Her earliest professional work included greeting cards and juvenile magazine illustrations , and her first book , Flower Fairies of the Spring , was published in 1923 . Similar books were published in the following decades . Barker was a devout Anglican , and donated her artworks to Christian fundraisers and missionary organizations . She produced a few Christian-themed books such as The Children ’ s Book of Hymns and , in collaboration with her sister Dorothy , He Leadeth Me . She designed a stained glass window for St. Edmund 's Church , Pitlake , and her painting of the Christ Child , The Darling of the World Has Come , was purchased by Queen Mary . Barker was equally proficient in water","Field hockey pitch = A hockey pitch is the playing surface for the game of field hockey . Historically , the game was played on natural turf ( grass ) but nowadays is predominantly played on an artificial turf . The transition onto artificial pitches came during the 1970s and was made mandatory for major competitions in 1976 . All the lines , markings and goal specifications are outlined by the International Hockey Federation in "" The Rules of Hockey "" . All line markings on the pitch form part of the area which they define . For example , a ball on the side line is still in the field of play ; a ball on the line of the penalty circle is in the penalty circle ; a foul committed over the 23-metre ( 25-yard ) line has occurred in the 23-metre area . A ball must completely cross a boundary line to be out of play , and a ball must wholly cross the goal line before a goal is scored . Due to the original formulation of the rules in England , the standard dimensions of a hockey pitch were ori","Transport in the Soviet Union = Transport in the Union of Soviet Socialist Republics ( USSR ) was an important part of the nation 's economy . The economic centralisation of the late 1920s and 1930s led to the development of infrastructure at a massive scale and rapid pace . Before the Soviet Union 's collapse in 1991 , there were a wide variety of modes of transport by land , water and air . However , because of government policies before , during and after the Era of Stagnation , investments in transport were low . By the late 1970s and early 1980s Soviet economists were calling for the construction of more roads to alleviate some of the strain from the railways and to improve the state budget . The Civil aviation industry , represented by Aeroflot , was the largest in the world , but inefficiencies plagued it until the USSR 's collapse . The road network remained underdeveloped , and dirt roads were common outside major cities . At the same time , the attendance of the few roads the","Amy Wyatt = Amy Marie Violet Wyatt is a fictional character from the British soap opera Emmerdale , played by Chelsea Halfpenny . She debuted on-screen during the episode airing on 8 October 2010 . Since her inception Amy has been central to main storylines and has been played as the serial 's rebellious teenager . Halfpenny has said she enjoys playing the character 's mischievous side . Amy 's unstable upbringing in children 's homes and foster care has impacted on her persona . She is a troubled character and protects herself by creating a hard exterior . Amy 's biggest fear is embarrassment and she aspires to lead a normal family life with her foster parents Val and Eric Pollard . As her storylines have developed she has grown close to the pair , yet rebelled at various stages . Charlie Har